In [1]:
import requests as r
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

In [2]:
#def getTradeInfo(html):
#    cik = html.find('rptOwnerCik'.lower()).text
#    name = html.find('rptOwnerName'.lower()).text
#    date = [pd.to_datetime(item.text.strip()) for item in html.findAll('transactionDate'.lower())]
#    sale = [item.text.strip() == 'D' for item in html.findAll('transactionAcquiredDisposedCode'.lower())]
#    amount = [float(item.text.strip()) for item in html.findAll('transactionShares'.lower())]
#    #ownedAfter = [float(item.text.strip()) for item in html.findAll('sharesOwnedFollowingTransaction'.lower())]
#    return cik, name, date, sale, amount#, ownedAfter

def getTradeInfo(html):
    cik = html.find('rptOwnerCik'.lower()).text
    name = html.find('rptOwnerName'.lower()).text
    dates, sales, amounts, rsus = [], [], [], []
    footnotes = html.find('footnotes')
    for transaction in html.findAll('nonDerivativeTransaction'.lower()):
        dates.append(pd.to_datetime( transaction.find('transactionDate'.lower()).text.strip() ))
        sales.append(transaction.find('transactionAcquiredDisposedCode'.lower()).text.strip() == 'D')
        
        shares = transaction.find('transactionShares'.lower())
        amounts.append(float( shares.text.strip() ))
        footnoteNm = shares.find('footnoteId'.lower()).attrs['id']
        footnoteTxt = footnotes.find(id=footnoteNm).text.lower()
        rsus.append(isRSUTrade(footnoteTxt))
    return cik, name, dates, sales, amounts, rsus

def isRSUTrade(text):
    rsuStrings = ['restricted', 'stock', 'units']
    return all(text.find(rsuString) > -1 for rsuString in rsuStrings)
    
#not presently needed
def isRealTrade(html):
    isRSU = isRSUTrade(html)
    isReal = (not isRSU, )
    if all(isReal):
        return True
    else:
        return False

def combineListValuedDictsViaAppend(dict0, dict1):
    combined = dict0.copy()
    for key in dict1:
        if key in combined:
            combined[key] += dict1[key]
        else:
            combined[key] = dict1[key]
    return combined

In [3]:
fNm = './company.idx'
companyNm = 'BOEING'
formNm = '4'
urlSuffixs = []
with open(fNm, 'r') as f:
    foundCompany = False
    for line in f:
        ls = line.split()
        if len(ls) > 3:
            if ls[0] == companyNm and ls[2] == formNm:
                urlSuffixs.append(ls[-1])
                foundCompany = True
            elif foundCompany is True:
                break #sorted by company

In [4]:
urlPrefix = 'https://www.sec.gov/Archives/'
urls = [urlPrefix + urlSuffix for urlSuffix in urlSuffixs]

In [5]:
#build as
#dict(([dates], [sales], [amounts], [ownedAfters]))[cik]
#later transform to
#dict(pd.dataframe[date, sale, amount, ownedAfter])[cik]
transactions = dict()
cikToName = dict()
cikToUrls = dict()
failedUrls = []
for i, url in enumerate(urls):
    response = r.get(url)
    try:
        response.raise_for_status()
    except r.exceptions.HTTPError or \
           r.exceptions.ConnectionError or \
           r.exceptions.MaxRetryError or \
           NewConnectionError or \
           OSError:
        print('error for url #{}'.format(i))
        failedUrls.append(url)
    else:
        html = BeautifulSoup(response.text, 'html.parser')
        cik, name, dates, sales, amounts, rsus = getTradeInfo(html)
        if cik not in transactions:
            cikToName[cik] = name
            cikToUrls[cik] = []
            transactions[cik] = {'dates': [],
                                 'sales': [],
                                 'amounts': [],
                                 'rsus': []}
        cikToUrls[cik].append(url)
        transactions[cik]['dates'] += dates
        transactions[cik]['sales'] += sales
        transactions[cik]['amounts'] += amounts
        transactions[cik]['rsus'] += rsus
    finally:
        sleep(1)

AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
print('len urls, len failedUrls, len transactions, n total transactions: {}, {}, {}, {}'.format(
    len(urls), len(failedUrls), len(transactions),
    sum(len(transactions[key]) for key in transactions)))
transactions

In [ ]:
#convert to dataframes & change amounts to account for transaction direction
for cik in transactions:
    transactions[cik] = pd.DataFrame(transactions[cik])
    transactions[cik]['amounts'] = transactions[cik]['amounts'] * (1. - 2. * transactions[cik]['sales'])

In [ ]:
print(cikToUrls['0001671300'])
transactions['0001671300']

In [ ]:
#todo:
##fix url fails; add loop to walk thru failed urls until completion?
##why sum(amounts) != ownedAfter?
##add ability to pull data from multiple years/quarters
##(1) entirely separate: walk thru index files, populate list with urls, plug into this code
##(2) outer loop: in new outer loop, read in an index file, then run this code as inner loop
##(3) ??
###I like the idea of having separate parts of code do separate things [ie. like "(1)" above]
##okay, let's go w/ "(1)" for now. 
##ideally want to have either a database or file storage system so that I don't have to grab
##all the files from SEC every time; should just update w/ any new ones
##(a) replace existing calls to online files with download of online files, calls to local files
##(b) 